In [4]:
import os
import io
from urllib.parse import urlencode
import pandas as pd
from astroquery.cadc import Cadc
from astropy.table import Table as A_Table
from IPython.display import Image, display, clear_output
from IPython.core.display import HTML
from astropy.io.votable.tree import VOTableFile, Resource, Table, Field
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time, TimeDelta
from cadcutils import net
from cadctap import CadcTapClient
import glue_jupyter as gj
import ipywidgets as widgets
from ipywidgets import interact
import base64

# setting the max displayed columns and rows
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 500)

# store the supernova catalog file into a dataframe
params = {'claimedtype': 'Ia', 'format': 'csv'}
Catalog_domain = 'https://api.sne.space/catalog'
query_str = urlencode(params)
url = '{}?{}&first'.format(Catalog_domain, query_str)
data = pd.read_csv(url)

# set up the input widgets and containers here
min_redshift = widgets.FloatSlider(description='Min Redshift',
                                   min=0.0,
                                   max=1.9,
                                   step=0.1,
                                   value=0.2,
                                   continuous_update=False)
min_redshift.style.handle_color = 'lightblue'
max_redshift = widgets.FloatSlider(description='Max Redshift',
                                   min=0.0,
                                   max=2.0,
                                   step=0.1,
                                   value=0.3,
                                   continuous_update=False)
max_redshift.style.handle_color = 'lightblue'
event = widgets.Dropdown(options=['SNLS', 'DES'],
                         description='Event')
days_before_maxd = widgets.IntSlider(description='Days Before Max Date',
                                     min=0,
                                     max=200,
                                     step=1,
                                     value=20,
                                     continuous_update=False,
                                     style={'description_width': 'initial'})
days_before_maxd.style.handle_color = 'moccasin'
days_after_maxd = widgets.IntSlider(description='Days After Max Date',
                                    min=0,
                                    max=200,
                                    step=1,
                                    value=100,
                                    continuous_update=False,
                                    style={'description_width': 'initial'})
days_after_maxd.style.handle_color = 'moccasin'
# use HBOX to group the widgets in one block
ui_catalog = widgets.HBox([min_redshift, max_redshift, event])
# use GridBox to group the widgets in one block, this one can be styled
ui_db = widgets.GridBox([days_before_maxd, days_after_maxd],
                        layout=widgets.Layout(width='100%',
                                              align_items='stretch',
                                              grid_template_columns='50% 50%'))
G_Type = widgets.Dropdown(options=['Scatter', 'Histogram', 'Table'],
                          description='View Data',
                          style={'description_width': 'initial'})
ui_graph = widgets.HBox([G_Type])
Obtain_Img = widgets.Dropdown(options=['Download Images', 'Display Images'],
                              description='Obtain Images',
                              style={'description_width': 'initial'})
ui_selection = widgets.HBox([Obtain_Img])


#
# Creating a votable file
# @param: fields_dictionary, df
# @return: no return type
#
def create_a_new_votable(field_dictionary, df):
    print("Creating the votable file for the tmp table.")
    # get total number of rows in the dataframe
    num_row = df.shape[0]
    votable = VOTableFile()
    resource = Resource()
    votable.resources.append(resource)
    table = Table(votable)
    resource.tables.append(table)
    list = []
    for key, value in field_dictionary.items():
        if (len(value) == 2):
            list.append(Field(votable,
                              name=key,
                              datatype=value[0],
                              arraysize=value[1]))
        elif (len(value) == 3):
            list.append(Field(votable,
                              name=key,
                              datatype=value[0],
                              arraysize=value[1],
                              xtype=value[2],
                              config={'version_1_2_or_later': True}))
        else:
            list.append(Field(votable, name=key, datatype=value))
    table.fields.extend(list)
    table.create_arrays(num_row)
    # replace the NaN value in satrt_date and end_date with 0.0 #

    for i in range(0, num_row):
        sn = sn_data.iloc[i]
        table.array[i] = (str(sn["event"]), str(sn["alias"]),
                          str(sn["catalog"]), str(sn["claimedtype"]),
                          sn["dec"], str(sn["discoverdate"]),
                          str(sn["discoverer"]), str(sn["download"]),
                          sn["ebv"], str(sn["host"]), str(sn["hostdec"]),
                          sn["hostoffsetang"], sn["hostoffsetdist"],
                          str(sn["hostra"]), str(sn["instruments"]),
                          sn["lumdist"], sn["maxabsmag"], sn["maxappmag"],
                          str(sn["maxdate"]), str(sn["name"]),
                          str(sn["photolink"]), sn["ra"], sn["radiolink"],
                          sn["redshift"], str(sn["references"]),
                          str(sn["spectralink"]), sn["velocity"],
                          sn["xraylink"], sn["start_date"], sn["end_date"],
                          [sn["start_date"], sn["end_date"]])
    votable.to_xml("tmp/new_out.xml")


# button waits on a onclick event
button = widgets.Button(description='Load Subsets',
                        layout=widgets.Layout(display='flex',
                                              flex_flow='column',
                                              align_items='stretch',
                                              width='50%'),
                        style=widgets.ButtonStyle(button_color='lightblue'))
out = widgets.Output()


#
# Check if a row is in any of the subsets, apply background color if yes
# @param: row(list)
# @return: background color
#
def background_color(row):
    color = ''
    # hardcode 11 colours
    color_list = ['#F780BF', '#C7E6A1', '#A1E6E2', '#FAFBA1',
                  '#FF9C33', '#CBC6C0', '#D8A56E', '#F7A480',
                  '#D798EE', '#80A0C8', '#FACCED']
    for item in range(0, len(out_data.subsets)):
        if row.values[3] in out_data.subsets[item]['Product ID']:
            color = color_list[item]
    return ['background-color: %s ' % color] * len(row.values)


#
# Interactive function, hinde and display different type of graphs
# @param: Scatter(widget), Histogram(widget)
# @return: no return type
#
def graphs(G_Type):
    print("")
    global out_data, results_df
    if (G_Type == 'Scatter'):
        scatter_viewer = app.scatter2d(x='Int. Time',
                                       y='Start Date',
                                       data=out_data,
                                       show=True)
        histogram_viewer = app.histogram1d(x='Int. Time',
                                           data=out_data,
                                           show=False)
    elif (G_Type == 'Histogram'):
        scatter_viewer = app.scatter2d(x='Int. Time',
                                       y='Start Date',
                                       data=out_data,
                                       show=False)
        histogram_viewer = app.histogram1d(x='Int. Time',
                                           data=out_data,
                                           show=True)
    else:
        if 'Polygon' in results_df.columns:
            results_df = results_df.drop(columns=['Polygon'])
        if (len(out_data.subsets)):
            subset_option = []
            subset_num_records = []
            items_auto = []
            color_list = ['#F780BF', '#C7E6A1', '#A1E6E2', '#FAFBA1',
                          '#FF9C33', '#CBC6C0', '#D8A56E', '#F7A480',
                          '#D798EE', '#80A0C8', '#FACCED']
            for i in range(len(out_data.subsets)):
                subset_option.append(out_data.subsets[i].label)
                n_records = len(out_data.subsets[i]['Product ID'])
                subset_num_records.append(n_records)
            for num in range(0, len(subset_option)):
                n_records = str(subset_num_records[num])
                t_tip = "number of records: {}".format(n_records)
                button = widgets.Button(description=subset_option[num],
                                        layout=widgets.Layout(flex='1 1 auto',
                                                              width='auto'),
                                        style=widgets.ButtonStyle(
                                            button_color=color_list[num]),
                                        tooltip=t_tip)
                items_auto.append(button)
            box_layout = widgets.Layout(display='flex',
                                        flex_flow='row',
                                        align_items='stretch',
                                        width='70%')
            box_auto = widgets.Box(children=items_auto,
                                   layout=box_layout)
            display(box_auto)
            display(results_df.style.apply(background_color,
                                           axis=1))
        else:
            display(results_df)


#
# Interactive function, used to process data, and create 5 widgets
# @param: min_redshift(widget), max_redshift, event,
# days_before_maxd, days_after_maxd
# @return: no return type
#
def processing_data(min_redshift, max_redshift, event,
                    days_before_maxd, days_after_maxd):
    global out_data, sn_data, click_flag, app, results_df, G_Type
    click_flag = 0
    button.click()  # triger the update subsets button to reaload
    if (min_redshift > max_redshift):
        print("\x1b[31m Min redshift is greater than max redshift \x1b[0m")
        return 0
    print("set constraints on the supernova catalog table")
    sn_data = data.copy()
    button.click()
    rshift = sn_data['redshift']
    sn_data = sn_data[(rshift > min_redshift) & (rshift < max_redshift)]
    sn_data = sn_data[sn_data['event'].str.startswith(event)]
    sn_data = sn_data[sn_data['discoverdate'] < sn_data['maxdate']]
    sn_data["ra"] = SkyCoord(sn_data["ra"], sn_data["dec"],
                             unit=(u.hourangle, u.deg)).ra.degree
    sn_data["dec"] = SkyCoord(sn_data["ra"], sn_data["dec"],
                              unit=(u.hourangle, u.deg)).dec.degree
    sn_data["maxdate"] = sn_data["maxdate"].str.replace('/', '-')
    print('Number of supernova: {}'.format(len(sn_data)))
    print("Calculating coords and start end date")
    for index, row in sn_data.iterrows():
        if (str(row["maxdate"]) != 'nan'):
            isot_maxdate = Time(row["maxdate"], format='isot')
            sd_jd = isot_maxdate - TimeDelta(days_before_maxd, format='jd')
            ed_jd = isot_maxdate + TimeDelta(days_after_maxd, format='jd')
            sn_data.at[index, 'start_date'] = sd_jd.mjd
            sn_data.at[index, 'end_date'] = ed_jd.mjd
        else:  # set place holder
            sn_data.at[index, 'start_date'] = 0.0
            sn_data.at[index, 'end_date'] = 0.1
    button.click()
    sn_data.dropna(subset=['maxdate'])

    field_dictionary = {
        "event": ["char", "*"],
        "alias": ["char", "*"],
        "catalog": ["char", "*"],
        "claimedtype": ["char", "*"],
        "dec": "double",
        "discoverdate": ["char", "*"],
        "discoverer": ["char", "*"],
        "download": ["char", "*"],
        "ebv": "double",
        "host": ["char", "*"],
        "hostdec": ["char", "*"],
        "hostoffsetang": "double",
        "hostoffsetdist": "double",
        "hostra": ["char", "*"],
        "instruments": ["char", "*"],
        "lumdist": "double",
        "maxabsmag": "double",
        "maxappmag": "double",
        "maxdate": ["char", "*"],
        "name": ["char", "*"],
        "photolink": ["char", "*"],
        "ra": "double",
        "radiolink": "double",
        "redshift": "double",
        "reference": ["char", "*"],
        "spectralink": ["char", "*"],
        "velocity": "double",
        "xraylink": "double",
        "start_date": "double",
        "end_date": "double",
        "start_end_date": ["double", "2", "interval"]
    }

    # create a votable file
    create_a_new_votable(field_dictionary, sn_data)

    # generate a CadcTapClient object
    anonSubject = net.Subject()
    client = CadcTapClient(anonSubject, resource_id='ivo://cadc.nrc.ca/argus')
    query = """SELECT Observation.observationURI AS "Preview",
    Observation.collection AS "Collection",
    Observation.observationID AS "Obs. ID",
    Plane.productID AS "Product ID",
    COORD1(CENTROID(Plane.position_bounds)) AS "RA (J2000.0)",
    COORD2(CENTROID(Plane.position_bounds)) AS "Dec. (J2000.0)",
    Plane.time_bounds_lower AS "Start Date",
    Observation.instrument_name AS "Instrument",
    Plane.time_exposure AS "Int. Time",
    Observation.target_name AS "Target Name",
    Plane.energy_bandpassName AS "Filter",
    Plane.calibrationLevel AS "Cal. Lev.",
    Observation.type AS "Obs. Type",
    Plane.energy_bounds_lower AS "Min. Wavelength",
    Plane.energy_bounds_upper AS "Max. Wavelength",
    Observation.proposal_id AS "Proposal ID",
    Observation.proposal_pi AS "P.I. Name",
    Plane.dataRelease AS "Data Release",
    AREA(Plane.position_bounds) AS "Field of View",
    Plane.position_bounds AS "Polygon",
    Plane.position_sampleSize AS "Pixel Scale",
    Plane.energy_resolvingPower AS "Resolving Power",
    Plane.time_bounds_upper AS "End Date",
    Plane.dataProductType AS "Data Type",
    Observation.target_moving AS "Moving Target",
    Plane.provenance_name AS "Provenance Name",
    Observation.intent AS "Intent",
    Observation.target_type AS "Target Type",
    Observation.target_standard AS "Target Standard",
    Observation.target_keywords AS "Target Keywords",
    Observation.sequenceNumber AS "Sequence Number",
    Observation.algorithm_name AS "Algorithm Name",
    Observation.proposal_title AS "Proposal Title",
    Observation.proposal_keywords AS "Proposal Keywords",
    Plane.position_resolution AS "IQ",
    Observation.instrument_keywords AS "Instrument Keywords",
    Observation.environment_tau AS "Tau",
    Plane.energy_transition_species AS "Molecule",
    Plane.energy_transition_transition AS "Transition",
    Observation.proposal_project AS "Proposal Project",
    Plane.energy_emBand AS "Band",
    Plane.provenance_version AS "Prov. Version",
    Plane.provenance_project AS "Prov. Project",
    Plane.provenance_runID AS "Prov. Run ID",
    Plane.provenance_lastExecuted AS "Prov. Last Executed",
    Plane.energy_restwav AS "Rest-frame Energy",
    Observation.requirements_flag AS "Quality",
    Plane.publisherID AS "Publisher ID" FROM caom2.Plane AS Plane
    JOIN caom2.Observation AS Observation
    ON Plane.obsID=Observation.obsID
    join tap_upload.tmptable as Tmp on
    (INTERSECTS(Tmp.start_end_date, Plane.time_bounds_samples)=1
    AND CONTAINS(POINT('ICRS',Tmp.ra,Tmp.dec),Plane.position_bounds)=1)
    WHERE (LOWER(Plane.energy_bandpassName) LIKE 'r%'
    AND collection='CFHT'
    AND calibrationLevel >= 2
    AND (Plane.quality_flag IS NULL OR Plane.quality_flag != 'junk'))
    ORDER BY time_bounds_lower ASC"""

    def_table = os.path.join('tmp', 'new_out.xml')
    print("fetching data from the database...")
    out_file = io.StringIO()
    client.query(query,
                 response_format='csv',
                 output_file=out_file,
                 tmptable='tmptable:' + def_table, timeout=30)

    # Hack: remove the triple quotes in the output csv file
    out_file.seek(0)
    results_df = pd.read_csv(out_file)
    count_row = len(results_df.index)
    cols = results_df.columns
    results_df.columns = [col.replace('"', '') for col in cols]
    results_df.to_csv('tmp/output_file.csv', index=False)
    print('Total row count: {}'.format(str(count_row)))

    # display data using glue
    app = gj.jglue()
    out_data = app.load_data('tmp/output_file.csv')
    print("displaying data.")
    button.click()
    click_flag = 1
    graph_output = widgets.interactive_output(graphs, {'G_Type': G_Type})
    display(ui_graph, graph_output)


# This interactive_output function allows me to modify the layout of widgets
output = widgets.interactive_output(processing_data,
                                    {'min_redshift': min_redshift,
                                     'max_redshift': max_redshift,
                                     'event': event,
                                     'days_before_maxd': days_before_maxd,
                                     'days_after_maxd': days_after_maxd})
# unlike interact(), interactive_output() does not display the widget itself
display(ui_catalog, ui_db, output)


#
# allow user to select a way to view the images using the provided urls
# @param: Obtain_Img(list widgest)
# @return: no return type
#
def select_method(Obtain_Img):
    # download all the urls as a text file
    if Obtain_Img == 'Download Images':
        txt = ""
        for key in url_dictionary:
            txt = txt + str(key) + '\n'
        b64 = base64.b64encode(txt.encode())
        payload = b64.decode()
        html = """<a download="testfile" href="data:text;base64, {payload}"
        target="_blank">Download Subset</a>"""
        html = html.format(payload=payload)
        display(HTML(html))
    # display the actual images when the number of images is smaller than 10.
    else:
        if len(url_dictionary) > 10:
            print("Can not display more than 10 images")
        else:
            for value in url_dictionary.values():
                print(value[1])
                img_url = str(value[0]),
                link = '<a href="{}">{}</a>'.format(img_url, img_url)
                display(HTML(link))
                display(Image(url=str(value[0]),
                              width=100,
                              height=100,
                              unconfined=True))


#
# interactive function, display and create a dataframe of selected data
# @param: subset(list widgest)
# @return: no return type
#
def preview_subset(subset):
    global url_dictionary, Obtain_Img
    lable = subset.split(' ')
    Id = int(lable[1]) - 1
    subset_data = {'Target Name': out_data.subsets[Id]['Target Name'],
                   'publisherID': out_data.subsets[Id]['Publisher ID'],
                   'Min. Wavelength': out_data.subsets[Id]['Min. Wavelength'],
                   'Start Date': out_data.subsets[Id]['Start Date'],
                   'Int. Time': out_data.subsets[Id]['Int. Time'],
                   'Product ID': out_data.subsets[Id]['Product ID'],
                   'Filter': out_data.subsets[Id]['Filter']}
    subset_df = pd.DataFrame(subset_data)
    cadc = Cadc()
    subset_table = A_Table.from_pandas(subset_df)
    url_dictionary = {}
    for idx in range(len(subset_table)):
        # The get_data_urls only accepts a table as input
        url = cadc.get_data_urls(subset_table[idx: idx + 1],
                                 include_auxiliaries=True)
        fz_url = next((u for u in url if '.fz' in u), None)
        jpg_url = next((u for u in url if '1024.jpg' in u), None)
        # build the dictionary
        if jpg_url:
            url_dictionary[fz_url] = [jpg_url, subset_df['Target Name'][idx]]
    display(subset_df)
    selection_output = widgets.interactive_output(select_method,
                                                  {'Obtain_Img': Obtain_Img})
    display(ui_selection, selection_output)


#
# function will get triggered by a on click event
# @param: no param
# @return: no return type
#
def on_button_clicked(_):
    # linking function with output
    with out:
        # what happens when we press the button
        clear_output()
        if (len(out_data.subsets) != 0 and click_flag == 1):
            list = []
            for i in range(len(out_data.subsets)):
                list.append(out_data.subsets[i].label)
            interact(preview_subset, subset=list)
        elif (len(out_data.subsets) == 0 and click_flag == 1):
            print('No subsets created')


# linking button and function
button.on_click(on_button_clicked)
# put the button and output into one container and display it
widgets.VBox([button, out])


# note: data.to_dataframe() convets a glue.Data object to pandas frame

GridBox(children=(IntSlider(value=20, continuous_update=False, description='Days Before Max Date', max=200, st…

Output()